## Data Manipulation

In this notebook, I will combine the data (1988-2020) downloaded from Kaggle (https://www.kaggle.com/datasets/zanjibar/100-million-data-csv) and ones downloaded (2021-2023) scraped from e-stat (https://www.e-stat.go.jp/en/stat-search/database?page=1&layout=normal&toukei=00350300&tstat=000001013141&result_page=1&metadata=1&data=1). 

I utilize PySpark features to handle large datasets. 

In [1]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [2]:
from pyspark.sql.types import StringType, LongType

import pyspark.sql.functions as F

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1684335481023_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
def translate(dictionary):
    return F.udf(lambda col: dictionary.get(col),
               StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Modify export data

In [4]:
data = spark.read.csv('s3://trade-final-project-bucket/dataset/trade_ex_2021_2023.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 12
Total Rows: 14634720
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)

In [6]:
data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+----------------+----------+----------------------+---------+------------------+----------+------------+----+-----+----------+
| _c0|       _c1|             _c2|       _c3|                   _c4|      _c5|               _c6|       _c7|         _c8| _c9| _c10|      _c11|
+----+----------+----------------+----------+----------------------+---------+------------------+----------+------------+----+-----+----------+
|null|cat01_code|統計品目表(輸出)|cat02_code|統計品目表の数量・金額|area_code|                国| time_code|時間軸(年次)|unit|value|annotation|
|   0| 010121000|       010121000|       110|                 単位2|    50103|      103_大韓民国|2022000000|      2022年|null| ＮＯ|      null|
|   1| 010121000|       010121000|       110|                 単位2|    50205|          205_英国|2022000000|      2022年|null| ＮＯ|      null|
|   2| 010121000|       010121000|       110|                 単位2|    50213|        213_ドイツ|2022000000|      2022年|null| ＮＯ|      null|
|   3| 010121000|       010121000|      

In [51]:
cat2 = [i for i in range(150, 501, 10)]
cat2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500]

In [8]:
data = data.withColumn("_c3", data._c3.cast('int')).filter(data._c3.isin(cat2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
cat2_month = {i:("0"+str(1+(e)//3)) for e,i in enumerate(range(150, 411, 10))}
cat2_month.update({i:("1"+str(e//3)) for e,i in enumerate(range(420, 501, 10))})
cat2_month

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{150: '01', 160: '01', 170: '01', 180: '02', 190: '02', 200: '02', 210: '03', 220: '03', 230: '03', 240: '04', 250: '04', 260: '04', 270: '05', 280: '05', 290: '05', 300: '06', 310: '06', 320: '06', 330: '07', 340: '07', 350: '07', 360: '08', 370: '08', 380: '08', 390: '09', 400: '09', 410: '09', 420: '10', 430: '10', 440: '10', 450: '11', 460: '11', 470: '11', 480: '12', 490: '12', 500: '12'}

In [10]:
data = data.withColumn("month",
              translate(cat2_month)("_c3"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
data[["_c1","_c3", "_c5", "_c7", "_c10", "month"]].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---+-----+----------+----+-----+
|      _c1|_c3|  _c5|       _c7|_c10|month|
+---------+---+-----+----------+----+-----+
|010121000|150|50103|2022000000|   0|   01|
|010121000|150|50205|2022000000|   0|   01|
|010121000|150|50213|2022000000|   0|   01|
|010121000|150|50213|2021000000|   0|   01|
|010121000|150|50234|2023000000|   0|   01|
|010121000|150|50234|2021000000|   0|   01|
|010121000|150|50304|2022000000|   0|   01|
|010121000|150|50601|2023000000|   0|   01|
|010121000|150|50601|2022000000|   0|   01|
|010121000|150|50601|2021000000|   0|   01|
|010121000|160|50103|2022000000|   0|   01|
|010121000|160|50205|2022000000|   0|   01|
|010121000|160|50213|2022000000|   0|   01|
|010121000|160|50213|2021000000|   4|   01|
|010121000|160|50234|2023000000|   0|   01|
|010121000|160|50234|2021000000|   0|   01|
|010121000|160|50304|2022000000|   0|   01|
|010121000|160|50601|2023000000|   0|   01|
|010121000|160|50601|2022000000|   0|   01|
|010121000|160|50601|2021000000|

In [12]:
data = (data.withColumn("year",F.substring("_c7", 1,4))
            .withColumn("country", F.substring("_c5", 3, 5))
            .withColumn("export_import", F.lit(1))
            .withColumn("custom", F.lit(0))
            .withColumnRenamed("_c1","hs9")
            .withColumnRenamed("_c10", "value")
       )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
data = data.select("_c3", "hs9", "export_import", "country", "custom", "year", "month", "value")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
data = data.withColumn("year_month", F.concat(data.year,data.month))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
data.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+-------------+-------+------+----+-----+-----+----------+
|_c3|      hs9|export_import|country|custom|year|month|value|year_month|
+---+---------+-------------+-------+------+----+-----+-----+----------+
|150|010121000|            1|    103|     0|2022|   01|    0|    202201|
+---+---------+-------------+-------+------+----+-----+-----+----------+
only showing top 1 row

In [52]:
cat2_code = {i:e%3 for e,i in enumerate(range(150, 501, 10))}
cat2_code

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{150: 0, 160: 1, 170: 2, 180: 0, 190: 1, 200: 2, 210: 0, 220: 1, 230: 2, 240: 0, 250: 1, 260: 2, 270: 0, 280: 1, 290: 2, 300: 0, 310: 1, 320: 2, 330: 0, 340: 1, 350: 2, 360: 0, 370: 1, 380: 2, 390: 0, 400: 1, 410: 2, 420: 0, 430: 1, 440: 2, 450: 0, 460: 1, 470: 2, 480: 0, 490: 1, 500: 2}

In [17]:
data = data.withColumn("type_of_value",
              translate(cat2_code)("_c3"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
data_pivot = (data.groupby("hs9","export_import","country","custom","year_month")
                  .pivot("type_of_value").agg(F.first("value")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
data_pivot.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------+-------+------+----------+---+---------+---------+
|      hs9|export_import|country|custom|year_month|  0|        1|        2|
+---------+-------------+-------+------+----------+---+---------+---------+
|000000110|            1|    107|     0|    202204|  0|        0|        0|
|000000190|            1|    105|     0|    202211|  0| 29825339|107465844|
|000000190|            1|    108|     0|    202105|  0|  2469734| 70599538|
|000000190|            1|    108|     0|    202202|  0|  2327056| 83981290|
|000000190|            1|    108|     0|    202207|  0|144532671| 85680996|
+---------+-------------+-------+------+----------+---+---------+---------+
only showing top 5 rows

In [20]:
data_pivot = (data_pivot.withColumnRenamed("0", "q1")
                        .withColumnRenamed("1", "q2")
                        .withColumnRenamed("2", "value")
             )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
data_pivot = data_pivot.select("year_month", "export_import", "country", "custom", "hs9", "q1", "q2", "value")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
data_pivot.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+-------+------+---------+---+---------+---------+
|year_month|export_import|country|custom|      hs9| q1|       q2|    value|
+----------+-------------+-------+------+---------+---+---------+---------+
|    202204|            1|    107|     0|000000110|  0|        0|        0|
|    202211|            1|    105|     0|000000190|  0| 29825339|107465844|
|    202105|            1|    108|     0|000000190|  0|  2469734| 70599538|
|    202202|            1|    108|     0|000000190|  0|  2327056| 83981290|
|    202207|            1|    108|     0|000000190|  0|144532671| 85680996|
+----------+-------------+-------+------+---------+---+---------+---------+
only showing top 5 rows

In [23]:
data_pivot.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4354872

### Modify import data

In [49]:
data = spark.read.csv('s3://trade-final-project-bucket/dataset/trade_im_2021_2023.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 12
Total Rows: 10168931
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)

In [26]:
data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+----------------+----------+----------------------+---------+------------+----------+------------+----+-----+----------+
| _c0|       _c1|             _c2|       _c3|                   _c4|      _c5|         _c6|       _c7|         _c8| _c9| _c10|      _c11|
+----+----------+----------------+----------+----------------------+---------+------------+----------+------------+----+-----+----------+
|null|cat01_code|統計品目表(輸入)|cat02_code|統計品目表の数量・金額|area_code|          国| time_code|時間軸(年次)|unit|value|annotation|
|   0| 010121100|       010121100|       110|                 単位2|    50205|    205_英国|2021000000|      2021年|null| ＮＯ|      null|
|   1| 010121100|       010121100|       110|                 単位2|    50207|207_オランダ|2022000000|      2022年|null| ＮＯ|      null|
|   2| 010121100|       010121100|       110|                 単位2|    50208|208_ベルギー|2022000000|      2022年|null| ＮＯ|      null|
|   3| 010121100|       010121100|       110|                 単位2|    50210|210_フラン

In [53]:
data = data.withColumn("_c3", data._c3.cast('int')).filter(data._c3.isin(cat2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
data = data.withColumn("month",
              translate(cat2_month)("_c3"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
data = (data.withColumn("year",F.substring("_c7", 1,4))
            .withColumn("country", F.substring("_c5", 3, 5))
            .withColumn("export_import", F.lit(2))
            .withColumn("custom", F.lit(0))
            .withColumnRenamed("_c1","hs9")
            .withColumnRenamed("_c10", "value")
       )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
data = data.select("_c3", "hs9", "export_import", "country", "custom", "year", "month", "value")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
data = data.withColumn("year_month", F.concat(data.year,data.month))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
data.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+-------------+-------+------+----+-----+-----+----------+
|_c3|      hs9|export_import|country|custom|year|month|value|year_month|
+---+---------+-------------+-------+------+----+-----+-----+----------+
|150|010121100|            2|    205|     0|2021|   01|    0|    202101|
+---+---------+-------------+-------+------+----+-----+-----+----------+
only showing top 1 row

In [59]:
data = data.withColumn("type_of_value",
              translate(cat2_code)("_c3"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
data_pivot2 = (data.groupby("hs9","export_import","country","custom","year_month")
                  .pivot("type_of_value").agg(F.first("value")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
data_pivot2 = (data_pivot2.withColumnRenamed("0", "q1")
                        .withColumnRenamed("1", "q2")
                        .withColumnRenamed("2", "value")
             )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
data_pivot2 = data_pivot2.select("year_month", "export_import", "country", "custom", "hs9", "q1", "q2", "value")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
data_pivot2.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+-------+------+---------+---+---+-----+
|year_month|export_import|country|custom|      hs9| q1| q2|value|
+----------+-------------+-------+------+---------+---+---+-----+
|    202207|            2|    103|     0|000000011|  0|  0|    0|
|    202311|            2|    143|     0|000000011|  0|  0|    0|
|    202110|            2|    147|     0|000000011|  0|  0|    0|
|    202202|            2|    147|     0|000000011|  0|  0|    0|
|    202304|            2|    147|     0|000000011|  0|  0|    0|
+----------+-------------+-------+------+---------+---+---+-----+
only showing top 5 rows

In [64]:
data_pivot2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3024240

### Merge two dataframes

In [65]:
data_all = data_pivot.union(data_pivot2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
data_all.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7379112

In [67]:
data_new = data_all.select([F.col(column).cast(LongType()).alias(column) for column in data_all.columns])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
data_new.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year_month: long (nullable = true)
 |-- export_import: long (nullable = false)
 |-- country: long (nullable = true)
 |-- custom: long (nullable = false)
 |-- hs9: long (nullable = true)
 |-- q1: long (nullable = true)
 |-- q2: long (nullable = true)
 |-- value: long (nullable = true)

In [69]:
output_path = "s3://trade-final-project-bucket/dataset/trade_2021_2023.parquet"
data_new.write.parquet(output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
data_check = spark.read.parquet("s3://trade-final-project-bucket/dataset/trade_2021_2023.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
print('Total Columns: %d' % len(data_check.dtypes))
print('Total Rows: %d' % data_check.count())
data_check.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 8
Total Rows: 7379112
root
 |-- year_month: long (nullable = true)
 |-- export_import: long (nullable = true)
 |-- country: long (nullable = true)
 |-- custom: long (nullable = true)
 |-- hs9: long (nullable = true)
 |-- q1: long (nullable = true)
 |-- q2: long (nullable = true)
 |-- value: long (nullable = true)